# Making Your Code Faster: Cython and parallel processing in the Jupyter Notebook  
  
### PyData DC 2016  
  
*Gustavo A. Patino*  
*Department of Biomedical Sciences*  
*Department of Neurology*  
*Oakland University William Beaumont School of Medicine*  
*Rochester, MI*  
  
patino@oakland.edu  
https://https://github.com/gapatino/Making-Your-Code-Faster-Cython-and-parallel-processing-in-the-Jupyter-Notebook  

**Problem:**  
The y=x^2 function can be approximated using its derivative y'=2x through the Euler method:  
y(n+1) = y(n) + (step \* y')  
The precision of the approximation depends on the step being very small
We want to find the step size that gives a difference < 1e-5 when comparing the values obtained  
using the y=x^2 formula and the Euler method after evaluating a million points  
*Note how a step size of 1 means we will evaluate values of x between 0 and 1000000, while a step size*  
*of 0.001 means that x ranges from 0 to 1000*

In [17]:
# Define the variable that will calculate values of y using both the y=x**2 formula and y'=2x using the Euler method
# Input is a step size for the Euler method; 10000 values of x will be evaluated
# Function returns the absolute difference between the last value of both sets of calculations

def errorEuler(step_size):
    # Range function only accepts integer values, to create list of values to evaluate will need to divide 
    # the list of 1000000 integers by (1/step_size)
    values0 = list(range(0,1000001))
    # Initialize variable that will keep results of x**2
    values_squared = []
    for index, value in enumerate(values0):
        values0[index] = value / (1/step_size)
        values_squared.append(values0[index]**2)
    # Calculate values of x**2 using Euler method and y'=2x
    # Start with initializing the variable that will contain those results
    values_euler = [(2*values0[0]*step_size)]
    for index, value in enumerate(values0[1:]):
        values_euler.append(values_euler[index-1]+(2*values0[index]*step_size))
    return abs(values_squared[-1] - values_euler[-1])


In [18]:
%%timeit
# Run the errorEuler function with different step sizes to find the one that returns a final difference lower 
# than 1e-5
# Initialize the difference value
difference0 = 1
# Initialize step size
step_size = 1
# Use a while loop to decrease step_size until we are below the desired difference
while difference0 > 1e-5:
    difference0 = errorEuler(step_size)
    step_size /= 10
print(step_size*10)

1.0000000000000003e-09
1.0000000000000003e-09
1.0000000000000003e-09
1.0000000000000003e-09
1 loop, best of 3: 9.27 s per loop


*Cython version*

In [19]:
%load_ext Cython

In [20]:
%%cython

# Make Cython version of errorEuler
def errorEuler_cython(float step_size):
    # Range function only accepts integer values, to create list of values to evaluate will need to divide 
    # the list of 1000000 integers by (1/step_size)
    cdef list values0 = list(range(0,1000001))
    # Initialize variable that will keep results of x**2
    cdef list values_squared = []
    for index, value in enumerate(values0):
        values0[index] = value / (1/step_size)
        values_squared.append(values0[index]**2)
    # Calculate values of x**2 using Euler method and y'=2x
    # Start with initializing the variable that will contain those results
    cdef list values_euler = [(2*values0[0]*step_size)]
    for index, value in enumerate(values0[1:]):
        values_euler.append(values_euler[index-1]+(2*values0[index]*step_size))
    return abs(values_squared[-1] - values_euler[-1])

In [21]:
%%timeit
# Run the errorEuler function with different step sizes to find the one that returns a final difference lower 
# than 1e-5
# Initialize the difference value
difference0 = 1
# Initialize step size
step_size = 1
# Use a while loop to decrease step_size until we are below the desired difference
while difference0 > 1e-5:
    difference0 = errorEuler_cython(step_size)
    step_size /= 10
print(step_size*10)

1.0000000000000003e-09
1.0000000000000003e-09
1.0000000000000003e-09
1.0000000000000003e-09
1 loop, best of 3: 5.7 s per loop


Maybe the first time was slow while doing the initial compilation?

In [22]:
%%timeit
# Run the errorEuler function with different step sizes to find the one that returns a final difference lower 
# than 1e-5
# Initialize the difference value
difference0 = 1
# Initialize step size
step_size = 1
# Use a while loop to decrease step_size until we are below the desired difference
while difference0 > 1e-5:
    difference0 = errorEuler_cython(step_size)
    step_size /= 10
print(step_size*10)

1.0000000000000003e-09
1.0000000000000003e-09
1.0000000000000003e-09
1.0000000000000003e-09
1 loop, best of 3: 5.68 s per loop


*Numpy version*

In [23]:
import numpy as np

In [24]:
def errorEuler_numpy(step_size):
    # Range function only accepts integer values, to create list of values to evaluate will need to divide 
    # the list of 1000000 integers by (1/step_size)
    values0 = np.arange(0,1000001) / (1/step_size)
    # Initialize variable that will keep results of x**2
    values_squared = values0**2
    # Calculate values of x**2 using Euler method and y'=2x
    # Start with initializing the variable that will contain those results
    values_euler = [(2*values0[0]*step_size)]
    for index, value in enumerate(values0[1:]):
        values_euler.append(values_euler[index-1]+(2*values0[index]*step_size))
    return abs(values_squared[-1] - values_euler[-1])

In [25]:
%%timeit
# Run the errorEuler function with different step sizes to find the one that returns a final difference lower 
# than 1e-5
# Initialize the difference value
difference0 = 1
# Initialize step size
step_size = 1
# Use a while loop to decrease step_size until we are below the desired difference
while difference0 > 1e-5:
    difference0 = errorEuler_numpy(step_size)
    step_size /= 10
print(step_size*10)

1.0000000000000003e-09
1.0000000000000003e-09
1.0000000000000003e-09
1.0000000000000003e-09
1 loop, best of 3: 8.44 s per loop


*Scipy version*

In [26]:
from scipy.integrate import ode

In [27]:
# define a function with the differential equation
# y=x**2  y'=2x  x(0)=0  y(0)=0
def EulerSquare(t, y): # Use t instead of x
    return 2*t

# initial conditions
y0 = 0
x0 = 0

# Create ODE object
solver = ode(EulerSquare)
solver.set_initial_value(y0, x0)
solver.t, solver.y

(0, array([ 0.]))

To solve the differential equation for a given value we simply use the integrate method with the value as input

In [28]:
solver.integrate(20)
solver.t, solver.y

(20.0, array([ 400.]))

Notice how solver.y is an np.ndarray  
solver.integrate() does not accept a list or np.array as input. For these reason we need a for loop to plug-in each  
of the 1000000 values we want to evaluate

In [29]:
# reset the solver to the intial conditions
solver.set_initial_value(y0, x0)

# Define the errorEuler function for scipy
def errorEuler_scipy(step_size):
    # Range function only accepts integer values, to create list of values to evaluate will need to divide 
    # the list of 1000000 integers by (1/step_size)
    values0 = np.arange(0,1000001) / (1/step_size)
    # Initialize variable that will keep results of x**2
    values_squared = values0**2
    # Calculate values of x**2 using Euler solver
    # Start with initializing the variable that will contain those results
    values_euler = solver.y
    for value in values0[1:]:
        solver.integrate(value)
        #values_euler = np.vstack((values_euler, solver.y))
    values_euler = solver.y
    return abs(values_squared[-1] - values_euler[-1])

In [30]:
%%timeit
solver = ode(EulerSquare)
solver.set_initial_value(y0, x0)
print(errorEuler_scipy(1))

0.00048828125


/Applications/anaconda/lib/python3.5/site-packages/scipy/integrate/_ode.py:869: UserWarning: vode: Illegal input detected. (See printed message.)
  'Unexpected istate=%s' % istate))


0.00048828125
0.00048828125
0.00048828125
1 loop, best of 3: 16.3 s per loop


In [31]:
%%timeit
solver.set_initial_value(y0, x0)
solver.set_integrator('dopri5') # Runge-Kutta method of order 4(5)
print(errorEuler_scipy(1))

0.0
0.0
0.0
0.0
1 loop, best of 3: 15.3 s per loop


*Numba version*

In [32]:
from numba import jit

In [33]:
# Make Numba version of errorEuler
errorEuler_numba=jit(errorEuler)

In [34]:
%%timeit
# Run the errorEuler function with different step sizes to find the one that returns a final difference lower 
# than 1e-5
# Initialize the difference value
difference0 = 1
# Initialize step size
step_size = 1
# Use a while loop to decrease step_size until we are below the desired difference
while difference0 > 1e-5:
    difference0 = errorEuler_numba(step_size)
    step_size /= 10
print(step_size*10)

1.0000000000000002e-07
1.0000000000000002e-07
1.0000000000000002e-07
1.0000000000000002e-07
1 loop, best of 3: 485 ms per loop


Note: Numba sometimes have problems compiling empty lists

In [35]:
# Make Numba version of errorEuler_numpy
errorEuler_numpy_numba=jit(errorEuler_numpy)

In [36]:
%%timeit
# Run the errorEuler function with different step sizes to find the one that returns a final difference lower 
# than 1e-5
# Initialize the difference value
difference0 = 1
# Initialize step size
step_size = 1
# Use a while loop to decrease step_size until we are below the desired difference
while difference0 > 1e-5:
    difference0 = errorEuler_numpy_numba(step_size)
    step_size /= 10
print(step_size*10)

1.0000000000000003e-09
1.0000000000000003e-09
1.0000000000000003e-09
1.0000000000000003e-09
1 loop, best of 3: 550 ms per loop


*Parallel processing version*  

Note: Need to start engines in the terminal:  
$ ipcluster start

In [37]:
from ipyparallel import Client

In [38]:
rc=Client() # Create ipyparallel.Client instance
v=rc[:]     # Create a view of the instance that includes all cores
rc.ids      # Returns identities of all the cores the instance has access to

[0, 1, 2, 3]

In [39]:
def errorEuler_parallel(step_size):
    # Range function only accepts integer values, to create list of values to evaluate will need to divide 
    # the list of 1000000 integers by (1/step_size)
    values0 = list(range(0,1000001))
    # Initialize variable that will keep results of x**2
    values_squared = []
    for index, value in enumerate(values0):
        values0[index] = value / (1/step_size)
        values_squared.append(values0[index]**2)
    # Calculate values of x**2 using Euler method and y'=2x
    # Start with initializing the variable that will contain those results
    values_euler = [(2*values0[0]*step_size)]
    for index, value in enumerate(values0[1:]):
        values_euler.append(values_euler[index-1]+(2*values0[index]*step_size))
    return abs(values_squared[-1] - values_euler[-1])

Asynchronous execution

In [40]:
%%timeit
# Run the errorEuler function with different step sizes to find the one that returns a final difference lower 
# than 1e-5
# Initialize the difference value
difference0 = 1
# Initialize step size, has to a list of 4 values so that errorEuler function is run with a different one in each
# of the 4 cores available
# Because of overhead in changing values of a list (since we are not using Numpy) the final print*10 is not feasible
# Better to initialize step_sizes with bigger values and first step in the while loop is to divide by 10000
step_size = [10000, 1000, 100, 1]  
# Use a while loop to decrease step_size until we are below the desired difference
while difference0 > 1e-5:
    for index, value in enumerate(step_size):
        step_size[index] = value/10000
    asynch_job=v.map(errorEuler_parallel, step_size) # Run function in each core with a different value from step_size
    asynch_results = asynch_job.get()                # Collect results from each core
    difference0 = min(asynch_results)
print(step_size)

[1e-08, 1e-09, 9.999999999999999e-11, 1e-12]
[1e-08, 1e-09, 9.999999999999999e-11, 1e-12]
[1e-08, 1e-09, 9.999999999999999e-11, 1e-12]
[1e-08, 1e-09, 9.999999999999999e-11, 1e-12]
1 loop, best of 3: 5.82 s per loop


Optimize the while loop with Numpy

In [41]:
%%timeit
# Run the errorEuler function with different step sizes to find the one that returns a final difference lower 
# than 1e-5
# Initialize the difference value
difference0 = 1
# Initialize step size, has to a list of 4 values so that errorEuler function is run with a different one in each
# of the 4 cores available
step_size = np.array([1, 0.1, 0.001, 0.0001])
# Use a while loop to decrease step_size until we are below the desired difference
while difference0 > 1e-5:
    asynch_job=v.map(errorEuler_parallel, step_size) # Run function in each core with a different value from step_size
    asynch_results = asynch_job.get()                # Collect results from each core
    difference0 = min(asynch_results)
    step_size /= 10000
print(step_size*10000)

[  1.00000000e-08   1.00000000e-09   1.00000000e-11   1.00000000e-12]
[  1.00000000e-08   1.00000000e-09   1.00000000e-11   1.00000000e-12]
[  1.00000000e-08   1.00000000e-09   1.00000000e-11   1.00000000e-12]
[  1.00000000e-08   1.00000000e-09   1.00000000e-11   1.00000000e-12]
1 loop, best of 3: 10.1 s per loop


Use Numpy also in the function

In [42]:
%px import numpy as np
%px import scipy.stats

In [43]:
def errorEuler_parallel_numpy(step_size):
    # Range function only accepts integer values, to create list of values to evaluate will need to divide 
    # the list of 1000000 integers by (1/step_size)
    values0 = np.arange(0,1000001) / (1/step_size)
    # Initialize variable that will keep results of x**2
    values_squared = values0**2
    # Calculate values of x**2 using Euler method and y'=2x
    # Start with initializing the variable that will contain those results
    values_euler = [(2*values0[0]*step_size)]
    for index, value in enumerate(values0[1:]):
        values_euler.append(values_euler[index-1]+(2*values0[index]*step_size))
    return abs(values_squared[-1] - values_euler[-1])

In [44]:
%%timeit
# Run the errorEuler function with different step sizes to find the one that returns a final difference lower 
# than 1e-5
# Initialize the difference value
difference0 = 1
# Initialize step size, has to a list of 4 values so that errorEuler function is run with a different one in each
# of the 4 cores available
step_size = np.array([1, 0.1, 0.001, 0.0001])
# Use a while loop to decrease step_size until we are below the desired difference
while difference0 > 1e-5:
    asynch_job=v.map(errorEuler_parallel_numpy, step_size) # Run function in each core w/ different val from step_size
    asynch_results = asynch_job.get()                      # Collect results from each core
    difference0 = min(asynch_results)
    step_size /= 10000
print(step_size*10000)

[  1.00000000e-08   1.00000000e-09   1.00000000e-11   1.00000000e-12]
[  1.00000000e-08   1.00000000e-09   1.00000000e-11   1.00000000e-12]
[  1.00000000e-08   1.00000000e-09   1.00000000e-11   1.00000000e-12]
[  1.00000000e-08   1.00000000e-09   1.00000000e-11   1.00000000e-12]
1 loop, best of 3: 4.96 s per loop


Synchronous execution

In [45]:
%%timeit
# Run the errorEuler function with different step sizes to find the one that returns a final difference lower 
# than 1e-5
# Initialize the difference value
difference0 = 1
# Initialize step size, has to a list of 4 values so that errorEuler function is run with a different one in each
# of the 4 cores available
step_size = np.array([1, 0.1, 0.001, 0.0001])
# Use a while loop to decrease step_size until we are below the desired difference
while difference0 > 1e-5:
    synch_results=v.map_sync(errorEuler_parallel_numpy, step_size) # Results don't need to be collected separately
    difference0 = min(synch_results)
    step_size /= 10000
print(step_size*10000)

[  1.00000000e-08   1.00000000e-09   1.00000000e-11   1.00000000e-12]
[  1.00000000e-08   1.00000000e-09   1.00000000e-11   1.00000000e-12]
[  1.00000000e-08   1.00000000e-09   1.00000000e-11   1.00000000e-12]
[  1.00000000e-08   1.00000000e-09   1.00000000e-11   1.00000000e-12]
1 loop, best of 3: 4.98 s per loop


In [47]:
# Make Numba version of errorEuler_parallel_numpy
errorEuler_parallel_numpy_numba=jit(errorEuler_parallel_numpy)

In [48]:
%%timeit
# Run the errorEuler function with different step sizes to find the one that returns a final difference lower 
# than 1e-5
# Initialize the difference value
difference0 = 1
# Initialize step size, has to a list of 4 values so that errorEuler function is run with a different one in each
# of the 4 cores available
step_size = np.array([1, 0.1, 0.001, 0.0001])
# Use a while loop to decrease step_size until we are below the desired difference
while difference0 > 1e-5:
    asynch_job=v.map(errorEuler_parallel_numpy_numba, step_size) # Run function in each core w/ different val from step_size
    asynch_results = asynch_job.get()                      # Collect results from each core
    difference0 = min(asynch_results)
    step_size /= 10000
print(step_size*10000)

[  1.00000000e-08   1.00000000e-09   1.00000000e-11   1.00000000e-12]
[  1.00000000e-08   1.00000000e-09   1.00000000e-11   1.00000000e-12]
[  1.00000000e-08   1.00000000e-09   1.00000000e-11   1.00000000e-12]
[  1.00000000e-08   1.00000000e-09   1.00000000e-11   1.00000000e-12]
1 loop, best of 3: 1.29 s per loop
